# PRÁCTICA 6.3: IMPLEMENTACIÓN DE UN TRANSFORMER


En esta práctica vamos a implementar y comparar una arquitectura Transformer Encoder hecha "a mano" frente a la implementación nativa de PyTorch, aplicando el modelo al dataset IMDB.

## 0. Objetivos 
1.  **Learned Positional Embeddings:** Sustitución de las funciones seno/coseno por embeddings aprendibles.
2.  **Arquitectura Modular:** Atención Multi-Cabezal y FeedForward.
3.  **Comparativa:** Switch para alternar entre `ManualTransformerEncoder` y `nn.TransformerEncoder`.
4.  **Aplicación:** Clasificación binaria (Positivo/Negativo).

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import math
import time
from datasets import load_dataset
import numpy as np

# Configuración de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Semilla para reproducibilidad
VOCAB_SIZE = 10000   # Tamaño de vocabulario supuesto
MAX_LEN = 200        # Longitud fija de secuencia (truncado/padding)
NUM_SAMPLES = 1000   # Número de reviews para entrenar (en IMDB son 25k)

Usando dispositivo: cuda


## 1. Carga de Datos (Integra tu código aquí)

**Ejercicio**: Vuelve a cargar los datos IMDB, tal y como hicimos en la práctica 6.1. Necesitarás traer la función que carga el vocabulario, el tokenizer (text_to_indices), y cargar los datos de IMDB.

**Requisitos de tus datos:** Define las siguientes variables
* `x_train`: Tensor de forma `(num_samples, max_len)` con índices de palabras.
* `y_train`: Tensor de forma `(num_samples)` con etiquetas 0 o 1.
* `x_test`: Tensor de forma `(num_samples, max_len)` con índices de palabras de test.
* `y_test`: Tensor de forma `(num_samples)` con etiquetas 0 o 1.

In [2]:
# Añade aquí tu código. Básate en la práctica 6.1 para ello.
# Usa los siguientes valores




In [3]:
# Solución
from collections import Counter
import numpy as np

# Construimos el vocabulario (las 1000 palabras más comunes)
def build_vocab(texts, max_words=1000):    
    counter = Counter()
    for text in texts:
        # Tokenización muy básica por espacios y minúsculas
        counter.update(text.lower().split())
    
    # Mapeo palabra -> índice (comenzando en 1, 0 reservado para padding)
    vocab = {word: i+1 for i, (word, _) in enumerate(counter.most_common(max_words))}
    return vocab

# Esto convierte las cadenas en listas de índices enteros.
def text_to_indices(texts, vocab, maxlen=20):
    indices_list = []
    for text in texts:
        # Convertir palabras a índices, ignorar las desconocidas
        seq = [vocab.get(word.lower(), 0) for word in text.split()]
        #seq = [idx for idx in seq if idx != 0] # Quitamos desconocidas (opcional)
        
        # Padding (Pre-padding)
        if len(seq) < maxlen:
            seq = [0] * (maxlen - len(seq)) + seq
        else: # Truncating 
            seq = seq[:maxlen]
            
        indices_list.append(seq)
    return np.array(indices_list)

In [4]:
# 1. Cargar el dataset desde Hugging Face
print("Descargando IMDB desde Hugging Face:")
dataset = load_dataset("imdb")

# El dataset tiene formato diccionario: {'train':, 'test':, 'unsupervised':}
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
test_texts = dataset['test']['text']
test_labels = dataset['test']['label']

# 2. Construir Vocabulario (Tokenización manual simple)
vocab = build_vocab(train_texts, VOCAB_SIZE-1)

# 3. Función para vectorizar (Texto -> Índices)
x_train = text_to_indices(train_texts, vocab, MAX_LEN)
x_test = text_to_indices(test_texts, vocab, MAX_LEN)
y_train = np.array(train_labels)
y_test = np.array(test_labels)
 

Descargando IMDB desde Hugging Face:


In [5]:
# Crear DataLoader estándar de PyTorch
import torch
from torch.utils.data import TensorDataset, DataLoader

BATCH_SIZE = 32
train_dataset = TensorDataset(torch.tensor(x_train).long(), torch.tensor(y_train).long())
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset = TensorDataset(torch.tensor(x_test).long(), torch.tensor(y_test).long())
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print(f"Datos cargados. Shape X: {x_train.shape}, Shape y: {y_train.shape}")

Datos cargados. Shape X: (25000, 200), Shape y: (25000,)


## 2. Componentes del Transformer  

Aquí definimos los bloques fundamentales de la arquitectura Transformer, solo en la parte de encoder. Hemos simplificado el **Positional Encoding** para usar embeddings aprendidos (`nn.Embedding`) en lugar de las funciones sinusoidales fijas.

La siguiente clase gestiona la etapa de *representación inicial*, un paso crucial debido a que la arquitectura Transformer procesa toda la secuencia en paralelo y, por tanto, carece de un sentido inherente del orden (a diferencia de las redes recurrentes o RNNs). Para solucionar esto, el modelo necesita fusionar la información semántica (qué es la palabra) con la información posicional (dónde está). Esto se logra mediante la suma vectorial elemento a elemento de dos embeddings: el `token_emb`, que captura el significado de la palabra, y el `pos_emb`, que codifica su índice dentro de la frase.

En este caso usaremos *Learned Positional Embeddings* en lugar de las funciones fijas de seno y coseno propuestas originalmente por Vaswani et al (ver Tema 7.2). Al utilizar nn.Embedding(max_len, d_model), tratamos las posiciones de manera análoga a las palabras: el modelo inicializa vectores aleatorios para la posición 0, para la posición 1, etc., y aprende sus valores óptimos mediante backpropagation durante el entrenamiento. Esto simplifica la arquitectura y permite que el modelo adapte su representación del espacio específicamente para la tarea de clasificación de texto.

In [6]:
class TransformerInput(nn.Module):
    """
    Capa de entrada: Suma Token Embeddings + Positional Embeddings.
    """
    def __init__(self, vocab_size, d_model, max_len, dropout=0.1):
        super().__init__()
        # Embedding de las palabras/tokens
        self.token_emb = nn.Embedding(vocab_size, d_model)
        # Positional Embedding aprendible (no sinusoidal)
        self.pos_emb = nn.Embedding(max_len, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x shape: (batch, seq_len)
        b, seq_len = x.shape
        
        # Crear vector de posiciones [0, 1, 2, ...]
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0).expand(b, -1)
        
        # Sumar y aplicar dropout
        x = self.token_emb(x) + self.pos_emb(positions)
        return self.dropout(x)


La clase `MultiHeadAttention` (abajo) constituye el núcleo del mecanismo de auto-atención. En lugar de realizar una única función de atención, proyecta la entrada en tres espacios diferentes **(Query, Key, Value)** y divide estas matrices en múltiples "cabezas" independientes. Esto permite al modelo capturar simultáneamente diferentes tipos de relaciones (como dependencias sintácticas vs. semánticas) en distintas posiciones de la secuencia. Vamos a implementar esto mediante operaciones matriciales: calcula los pesos de atención (Scaled Dot-Product), aplica la máscara para ignorar el padding y finalmente concatena los resultados de todas las cabezas para proyectarlos de vuelta a la dimensión original.

Por su parte, la clase `FeedForward` es una red neuronal densa que se aplica "posición a posición" (es decir, a cada palabra por separado e idénticamente). Funciona como una capa de procesamiento que toma la información contextualizada por la atención y la transforma: expande la dimensión de los datos (típicamente multiplicando d_model por 4), aplica una no-linealidad (ReLU) y luego los comprime de nuevo. Su objetivo es añadir profundidad y capacidad de cómputo no lineal al bloque, permitiendo que el modelo procese las características extraídas por la atención antes de pasar al siguiente bloque.

In [7]:

class MultiHeadAttention(nn.Module):
    """
    Implementación manual de la Atención Multi-Cabezal.
    """
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0
        
        # Dimensión de los vectores en el modelo
        self.d_model = d_model
        # Dimensión de cada cabeza
        self.d_k = d_model // num_heads
        self.num_heads = num_heads
        
        # Q, K, V son del tamaño del modelo, pero
        # luego lo dividiremos en cabezas
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.out_proj = nn.Linear(d_model, d_model)
        
    def forward(self, x, mask=None):
        batch_size = x.size(0)
        
        # Proyecciones y reshape para separar cabezas. La forma final es (Batch, Heads, Seq, d_k)
        # Al conseguir esa forma, podemos hacer la multiplicación de todas las cabezas a la vez
        # con solo un matmul, lo cual es más eficiente
        Q = self.q_linear(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.k_linear(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.v_linear(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        # Fíjate que la entrada x pasa en paralelo por Q, K y V, ahora multplicamos todo
        # Atención escalar escalada
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Cuando la frase es más corta que la longitud de entrada, podemos hacer padding,
        # dejando los huecos con valores muy pequeños, para que la atención no asocie las
        # palabras con los huecos
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
            
        attn_weights = torch.softmax(scores, dim=-1)
        attn_output = torch.matmul(attn_weights, V)
        
        # Unificar cabezas
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        return self.out_proj(attn_output)

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model)
        )
    def forward(self, x):
        return self.net(x)

## 3. Implementaciones del Encoder: Manual vs Nativa

Aquí creamos dos clases distintas:
1.  **Manual:** Usa las clases anteriores, demostrando que podemos crear un modelo Transformer de forma manual. La implementación manual (`ManualTransformerEncoder`) reconstruye la arquitectura fielmente siguiendo el esquema original de Vaswani et al. La clase ManualEncoderLayer define la unidad fundamental del modelo, orquestando las dos subcapas principales: la atención y la red feed-forward. Un detalle crítico en este código es la aplicación explícita de las conexiones residuales y la normalización (el patrón norm(x + subcapa)), que son esenciales para evitar el desvanecimiento del gradiente en redes profundas. El encoder completo no es más que una pila secuencial (`nn.ModuleList`) de estas capas idénticas, donde la salida de una sirve directamente como entrada de la siguiente, refinando progresivamente la representación de la secuencia.


2.  **Nativa:** Usa `nn.TransformerEncoder` de PyTorch, que ya implementa todo lo anterior, y está optimizada en C++/CUDA. `NativeTransformerEncoder` delega toda esta lógica en las clases optimizadas de PyTorch. Al utilizar nn.TransformerEncoderLayer, accedemos a kernels escritos en C++ y CUDA que son significativamente más rápidos y eficientes en el uso de memoria que nuestra versión en Python puro. Un punto técnico a destacar aquí es el parámetro batch_first=True; por defecto, los Transformers en PyTorch esperan los datos en formato (Seq, Batch, Dim), pero al activar esta bandera forzamos al modelo a trabajar con el formato estándar (Batch, Seq, Dim), alineándolo con nuestros dataloaders y simplificando la gestión de dimensiones sin necesidad de transponer tensores manualmente.

In [8]:
# --- OPCIÓN 1: MANUAL ---

# Una capa
class ManualEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.attn = MultiHeadAttention(d_model, num_heads)
        self.ff = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model) # distinto a batchnorm
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        # Atención + Residual + Norm
        attn_out = self.attn(x, mask)
        x = self.norm1(x + self.dropout(attn_out))
        # FF + Residual + Norm
        ff_out = self.ff(x)
        x = self.norm2(x + self.dropout(ff_out))
        return x

# El Transformer
class ManualTransformerEncoder(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, num_layers, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([
            ManualEncoderLayer(d_model, num_heads, d_ff, dropout) 
            for _ in range(num_layers) # replicamos un número de capas
        ])
    
    def forward(self, x, mask=None):
        for layer in self.layers:  # iteramos las capas
            x = layer(x, mask)
        return x

In [9]:
# --- OPCIÓN 2: NATIVA (PYTORCH) ---
class NativeTransformerEncoder(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, num_layers, dropout=0.1):
        super().__init__()
        # batch_first=True es clave para no transponer dimensiones
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=num_heads, 
            dim_feedforward=d_ff, 
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
    def forward(self, x, mask=None):
        # src_key_padding_mask espera (Batch, Seq) con True donde hay padding
        # Aquí lo pasamos simple por compatibilidad con la manual
        return self.transformer(x)

## 4. El Modelo Clasificador

Este modelo envuelve todo. Tiene un parámetro `use_native` en el constructor. Si es `True`, usa la capa optimizada de PyTorch; si es `False`, usa nuestra implementación manual.

La clase `IMDBTransformerClassifier` actúa como el arquitecto final que integra todos los componentes para resolver la tarea específica de análisis de sentimiento. Su responsabilidad principal es gestionar la transición de una secuencia de palabras a una única predicción de clase; para ello, toma la salida enriquecida del encoder (una secuencia de vectores contextualizados) y aplica una operación de Global Mean Pooling (promedio), colapsando toda la información de la frase en un único vector representativo. Este "vector resumen" es el que finalmente se proyecta a través de la capa lineal (classifier_head) para obtener los logits que determinarán si la reseña es positiva o negativa.

In [10]:
class IMDBTransformerClassifier(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, d_ff, num_layers, max_len, num_classes=2, use_native=False):
        super().__init__()
        
        # 1. Embeddings (Común)
        self.input_layer = TransformerInput(vocab_size, d_model, max_len)
        
        # 2. Encoder (Seleccionable)
        self.use_native = use_native
        if use_native:
            self.encoder = NativeTransformerEncoder(d_model, num_heads, d_ff, num_layers)
        else:
            self.encoder = ManualTransformerEncoder(d_model, num_heads, d_ff, num_layers)
            
        # 3. Clasificador
        self.classifier_head = nn.Linear(d_model, num_classes)

    def forward(self, x, mask=None):
        # x: (Batch, Seq_Len)
        x = self.input_layer(x)
        
        # Paso por Encoder
        x = self.encoder(x, mask)
        
        # POOLING: Promediar toda la secuencia para obtener un solo vector por review
        x = x.mean(dim=1)
        
        logits = self.classifier_head(x)
        return logits

## 5. Entrenamiento y Comparación

Vamos a entrenar dos modelos idénticos en hiperparámetros, pero cambiando el motor del encoder. Usaremos nuestra implementación manual, y el basado en la capa ya predefinida en PyTorch.

In [15]:
def train_and_evaluate(model_name, use_native_encoder, epochs=3):
    print(f"\n--- Entrenando Modelo: {model_name} ---")
    
    # Instanciar modelo
    model = IMDBTransformerClassifier(
        vocab_size=VOCAB_SIZE,
        d_model=64,       # Dimensiones ajustadas para demostración
        num_heads=4,
        d_ff=128,
        num_layers=2,
        max_len=MAX_LEN,
        use_native=use_native_encoder
    ).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    start_time = time.time()
    
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0
        
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            
            # Forward
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            
            # Backward
            loss.backward()
            optimizer.step()
            
            # Métricas
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
            
        avg_loss = total_loss / len(train_loader)
        acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Acc: {acc:.2f}%")
        
    elapsed = time.time() - start_time
    print(f"Tiempo Total ({model_name}): {elapsed:.2f} segundos")

    return model

# 1. Ejecutar Implementación Manual
modelManual = train_and_evaluate("MANUAL", epochs=10, use_native_encoder=False)

# 2. Ejecutar Implementación Nativa (PyTorch)
modelNative = train_and_evaluate("NATIVA (PyTorch)", epochs=10, use_native_encoder=True)


--- Entrenando Modelo: MANUAL ---
Epoch 1/10 | Loss: 0.5583 | Acc: 69.45%
Epoch 2/10 | Loss: 0.3958 | Acc: 82.34%
Epoch 3/10 | Loss: 0.3303 | Acc: 85.86%
Epoch 4/10 | Loss: 0.2865 | Acc: 88.14%
Epoch 5/10 | Loss: 0.2553 | Acc: 89.76%
Epoch 6/10 | Loss: 0.2250 | Acc: 90.91%
Epoch 7/10 | Loss: 0.2017 | Acc: 92.09%
Epoch 8/10 | Loss: 0.1774 | Acc: 93.08%
Epoch 9/10 | Loss: 0.1601 | Acc: 93.90%
Epoch 10/10 | Loss: 0.1412 | Acc: 94.65%
Tiempo Total (MANUAL): 42.75 segundos

--- Entrenando Modelo: NATIVA (PyTorch) ---
Epoch 1/10 | Loss: 0.5724 | Acc: 68.47%
Epoch 2/10 | Loss: 0.4098 | Acc: 81.14%
Epoch 3/10 | Loss: 0.3360 | Acc: 85.62%
Epoch 4/10 | Loss: 0.2906 | Acc: 87.90%
Epoch 5/10 | Loss: 0.2577 | Acc: 89.54%
Epoch 6/10 | Loss: 0.2227 | Acc: 91.01%
Epoch 7/10 | Loss: 0.1972 | Acc: 92.22%
Epoch 8/10 | Loss: 0.1766 | Acc: 93.16%
Epoch 9/10 | Loss: 0.1525 | Acc: 94.06%
Epoch 10/10 | Loss: 0.1388 | Acc: 94.76%
Tiempo Total (NATIVA (PyTorch)): 46.97 segundos


**Ejercicio**: Evalúa el modelo `modelNative` sobre el conjunto de test.

In [ ]:
# Haz un recorrido sobre el conjunto de test, y calcula el loss y el accuracy. 
# Puedes basarte en el bucle de entrenamiento, pero teniendo en cuenta lo necesario
# para usar el modelo en modo evaluación

In [21]:
# Solución
total_loss=0
total=0
correct=0
for batch_x, batch_y in test_loader:
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    modelNative.eval()
    with torch.no_grad():
        # Forward
        outputs = modelNative(batch_x)
        loss = nn.CrossEntropyLoss()(outputs, batch_y)
    
        # Métricas
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
    
avg_loss = total_loss / len(test_loader)
acc = 100 * correct / total
print(f"Loss: {avg_loss:.4f} | Acc: {acc:.2f}%")

Loss: 0.5125 | Acc: 83.26%


## 6. Conclusiones

Has visto algunos conceptos básicos para implementar Transformers. Por un lado, los Q, K y V se implementan con un solo tensor, pero luego se dividen para simular el multi-cabezal. Por otro lado, hemos visto que PyTorch ya trae capas de auto-atención, que nos puede servir para montar un modelo desde cero. Hemos evaluado las dos formas sobre IMDB, usando el tokenizador de la práctica 6.1.